In [1]:
from pyspark.sql import SparkSession
import os

In [2]:
spark = (
    SparkSession
    .builder
    .appName('DibimbingStreaming')
    .master('spark://dataeng-spark-master:7077')
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2')
    .getOrCreate()
)
spark.sparkContext.setLogLevel('WARN')


In [3]:
spark

In [18]:
spark.stop()

In [4]:
streaming = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "test-topic")
    .option("startingOffsets", "latest")
    .load()
)

In [5]:
streaming.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
from pyspark.sql.types import *

schema = StructType(
    [
        StructField("order_id", StringType(), True),
        StructField("customer_id", IntegerType(), True),
        StructField("furniture", StringType(), True),
        StructField("color", StringType(), True),
        StructField("price", IntegerType(), True),
        StructField("ts", IntegerType(), True),
    ]
)

In [2]:
import pyspark.sql.functions as F

json_df = streaming.selectExpr('CAST(value AS STRING) as value')
json_expanded_df = json_df.withColumn("value", F.from_json('value', schema)).select('value.*') 
json_expanded_df = json_expanded_df.withColumn('ts', F.from_unixtime('ts').cast('timestamp'))
json_expanded_df.printSchema()

NameError: name 'streaming' is not defined

In [1]:
from pyspark.sql.functions import *

# Aggregate data with group window 1 day
total_per_day = (
    json_expanded_df
    .withWatermark('ts', '2 hours')
    .groupBy(window('ts', '1 day').alias('day'))
    .agg(
        sum('price').alias('total_price'),
        count('order_id').alias('total_order')
    )
)

NameError: name 'json_expanded_df' is not defined

In [17]:
# Write data stream to parquet

(
    total_per_day
    .writeStream
    .format('parquet')
    .trigger(processingTime='1 minutes')
    .outputMode('append')
    .partitionBy('day')
    .option('checkpointlocation', '/home/jovyan/')
    .option('path', '/home/jovyan/data')
    .start() 
    .awaitTermination()
)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 